In [1]:
from __future__ import print_function
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate, Reshape
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.utils import plot_model
from keras.models import Model
from keras.optimizers import adadelta as ada
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import copy
import pydot
from keras.utils import multi_gpu_model
from keras import backend as K



import gc

import time



%matplotlib inline

Using TensorFlow backend.
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\mrathbun2018\.con

In [2]:
hdf5_path = 'Data/dataset.hdf5'
subtract_mean = True
hdf5_file = h5py.File(hdf5_path, "r")
if subtract_mean:
    mm = hdf5_file["train_mean"][0, ...]
    mm = mm[np.newaxis, ...]
data_num = hdf5_file["train_flow"].shape[0]
    

    
num_classes = 2
epochs = 30

flow_rows, flow_cols = 298, 17

    

x_train = hdf5_file["train_flow"][:,...]
if subtract_mean:
    x_train -= mm

y_train = hdf5_file["train_labels"][:, ...]

hdf5_file.close()
    
hdf5_path = 'Data/dataset-IoT.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")

x_test = hdf5_file["IoT_flow"][:,...]
if subtract_mean:
    x_test -= mm

y_test = hdf5_file["labels"][:, ...]

hdf5_file.close()





class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

packets = 230
x_train = x_train[:,:packets,:,:]
x_test = x_test[:,:packets,:,:]

input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [3]:
AccScores = []
Times = []
LossScores = []
#AucScores = []
#F1Scores = []
#PrecisionScores = []
t2 = time.time()
for i in range (1,50):
    batch_size = 1024
    epochs = 30
    filters= 32
    kernel_size= (3,3)
    activations= "sigmoid"
    pool_size= (2,2)
    dropout = 0.29104403739531765
    lr = 0.001433773291970261
    rmsprop = keras.optimizers.RMSprop(lr=lr)
    optim = rmsprop
    layers = 2
    model = Sequential()

    model.add(Conv2D(filters, kernel_size=kernel_size,activation=activations, input_shape=input_shape,padding = "same"))
    for i in range(layers-1):
        model.add(Conv2D(filters,kernel_size=kernel_size, activation=activations, padding = "valid"))
        
    model.add(MaxPooling2D(pool_size=(pool_size)))
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(num_classes, activation='softmax'))
    #model.summary()
    try:
        model = multi_gpu_model(model, gpus = 4)
    except:
        pass
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    t0 = time.time()
    model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=0, class_weight=class_weights, shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=0)
    t1 = time.time()
    total_run = t1-t0
    Times.append(total_run)
    print("%s: %.5f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.5f%%" % (model.metrics_names[0], scores[0]))
    print('Fit and Evaluate Time:',total_run)
    AccScores.append(scores[1] * 100)
    LossScores.append(scores[0])
    del model
    gc.collect()
    K.clear_session()
    gc.collect()
    
print("%.5f%% (+/- %.5f%%)" % (np.mean(AccScores), np.std(AccScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(LossScores), np.std(LossScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(Times), np.std(Times)))
t3 = time.time()
total_time = t3-t2
print('Time to do everything:',total_time)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

accuracy: 88.41274%
loss: 0.26477%
Fit and Evaluate Time: 80.05391764640808
accuracy: 82.94164%
loss: 2.62342%
Fit and Evaluate Time: 72.85394477844238
accuracy: 84.07522%
loss: 0.29044%
Fit and Evaluate Time: 73.18490076065063
accuracy: 82.94164%
loss: 2.62342%
Fit and Evaluate Time: 72.30545663833618
accuracy: 83.22622%
loss: 0.30168%
Fit and Evaluate Time: 73.29101181030273
accuracy: 86.33292%
loss: 0.29238%
Fit and Evaluate Time: 73.24856519699097
accuracy: 88.69021%
loss: 0.27188%
Fit and Evaluate Time: 73.0212914943695
accuracy: 87.73448%
loss: 0.26379%
Fit and Evaluate Time: 73.00745034217834
accuracy: 87.90049%
loss: 0.26024%
Fit and Evaluate Time: 69.65710735321045
accuracy: 85.07127%
loss: 0.31406%
Fit and Evaluate Time: 72.42548274993896
accuracy: 87.95741%
loss: 0.26546%
Fit and Evaluate Time: 75.33400177955627
accuracy: 87.91709%
loss: 0.26089%
Fit and Evaluate Time: 71.08365488

In [4]:
AccScores

[88.41273784637451,
 82.94163942337036,
 84.07522439956665,
 82.94163942337036,
 83.22622179985046,
 86.33291721343994,
 88.69020938873291,
 87.73448467254639,
 87.90048956871033,
 85.07126569747925,
 87.95740604400635,
 87.91708946228027,
 87.61590719223022,
 87.85543441772461,
 87.70365715026855,
 87.27678060531616,
 82.94163942337036,
 83.28313827514648,
 87.0609700679779,
 88.29653263092041,
 87.91471719741821,
 86.95899844169617,
 88.05938363075256,
 88.10444474220276,
 83.27127695083618,
 87.76057362556458,
 82.94163942337036,
 87.26492524147034,
 87.22698092460632,
 87.43093013763428,
 87.03251481056213,
 87.13685870170593,
 87.84831762313843,
 88.6072039604187,
 87.16294765472412,
 87.05148696899414,
 87.63013482093811,
 88.02143931388855,
 82.9392671585083,
 87.7273678779602,
 88.54791522026062,
 87.56373524665833,
 87.78428435325623,
 84.59933400154114,
 87.69654035568237,
 86.51552200317383,
 87.55424618721008,
 83.82858633995056,
 82.94163942337036]

In [ ]:
LossScores

In [ ]:
Times